In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy
import pandas as pd
import time
from itertools import product, groupby
import math
import scipy.stats as stats 

In [2]:
from mpmath import mp
mp.dps = 50

In [3]:
def prior_kernel(s,t):
    return np.exp(-(t-s)**2)+0.0001*(s==t)

def runs(a_list):
    runs_zeros = len([k for k, g in groupby(a_list) if k == 0])
    runs_ones = len([k for k, g in groupby(a_list) if k == 1])
    return runs_ones + runs_zeros

def tr(actions):
    if len(actions)<100:
        return 1
    else:
        return 1-0.02*(100-sum(actions[-100:-1]))/100

In [4]:
class posterior:
    def __init__(self, T, T0, x, y, K,sigma):
        self.T = T
        self.x = x
        self.y = y
        self.n = y.shape[0]
        self.g = T.shape[0]
        self.T0 = np.array(T0)
        self.sigma = sigma
        self.K = K
        self.K0 = self.get_K0()
    
    def get_K(self):
        return self.K
    
    def getter(self):
        return self.K0
    
    def get_K0(self):
        tau = np.where(self.T0==1)[0]
        K_tau=self.K[:, tau]
        K_tau_tau = self.K[tau[:, None], tau]
        return self.K- np.dot(np.dot(K_tau, np.linalg.inv(K_tau_tau)), K_tau.T)
        
    def Lx0(self,t):   
        L = np.zeros(self.n)    
        for i in np.arange(0,self.n):
            L[i] = np.trapz(self.K0[t, :] * self.x[i,:], self.T)
        return L
    
    def likelihood(self): #p(Y|T0) 
        m_matrix = self.M()
        first = np.linalg.det(m_matrix) ** (-0.5)

        
        inv = np.linalg.inv(m_matrix)
        s11 = np.dot(np.dot(np.transpose(np.ones(self.n)),inv),np.ones(self.n))
        
        syy = np.dot(np.dot(np.transpose(self.y),inv),self.y)
        s1y = np.dot(np.dot(np.transpose(np.ones(self.n)),inv),self.y)
        b = 0.5*(syy-s1y**2/s11)
            
        return first/np.sqrt(s11)*(b**(-0.5*(self.n-1)))
    
    def R0(self,i, j):
        Ri = np.trapz(self.K0[:, :] * self.x[i, :], self.T, axis=1)
        Rij = np.trapz(Ri * self.x[j, :],self.T)
        return Rij

    def M(self):
        return np.array([[self.R0(i, j) for j in range(self.n)] for i in range(self.n)])+np.identity(self.n)

    def posterior_mean_cov(self):
        M_mat = self.M()
        inv = np.linalg.inv(M_mat)
        S11 = np.dot(np.dot(np.transpose(np.ones(M_mat.shape[0])),inv),np.ones(M_mat.shape[0]))
        SY1 = np.dot(np.dot(np.transpose(self.y),inv),np.ones(M_mat.shape[0]))
        
        def m(t):
            mean = np.dot(np.dot(np.transpose(self.Lx0(t)), inv), (self.y - (SY1 / S11) * np.ones(len(self.y))))
            return mean
        def Kstar(s,t):
            cov = (self.sigma**2)*(self.K0[s,t] - np.dot(np.dot(np.transpose(self.Lx0(s)),inv),self.Lx0(t)))
            return cov
        
        m = np.fromfunction(np.vectorize(lambda t: m(t)), (self.g,), dtype=int)
        C = np.fromfunction(np.vectorize(lambda t, s: Kstar(s,t)), (self.g,self.g), dtype=int)
        return m,C
    
    def only_mean(self):
        inv = np.linalg.inv(self.M())
        def m(t):
            mean = np.dot(np.dot(np.transpose(self.Lx0(t)),inv),self.y)
            return mean
        m = np.fromfunction(np.vectorize(lambda t: m(t)), (self.g,), dtype=int)
        return m
    
    def fitted_values(self):
        m = self.only_mean()
        return np.fromfunction(np.vectorize(lambda i: np.trapz(self.x[i,:]*m,self.T)), (self.n,), dtype=int)

In [5]:
class T0_est:  
    def __init__(self, T, x, y,K,sigma,alpha,n_sim=1000,burn=0):
        self.T= T
        self.x=x
        self.y=y
        self.sigma=sigma
        self.alpha=alpha
        self.n_sim=n_sim
        self.burn=burn
        self.n = y.shape[0]
        self.g = T.shape[0]
        self.K=K
        self.count = 0
        
        self.T0_init = np.array([0, 1] * (self.g // 2))
        #self.T0_init = np.concatenate((np.ones(10),np.zeros(100),np.ones(18)))
        self.action = []
    
    def chain_builder(self):
        chain = [self.T0_init]
        for i in range(self.n_sim):
            curr=chain[-1]
            chain.append(self.next_T0(curr))
            print('Current Number of Points in T0: '+str(sum(chain[-1])))
        return chain[self.burn:]
    
        
    def next_T0(self,current): 
        new = self.candidate(current)
        rate=self.acceptance_rate(current,new)
        print('Acceptance Rate = '+str(rate)+'\n')
        treshold = tr(self.action)
        
        self.count = self.count + 1
        print(str(self.count)+'/'+str(self.n_sim) + ' in findind T0...\n')
        if rate>=treshold:
            self.action.append(1)
            return new
        
        else:
            self.action.append(0)
            return current
        
    def candidate(self,old):
        new=old.copy()
        coin = np.random.uniform(0,1,1)
        
        if coin<0.2: #split
            choices = [i for i in range(1,len(old) - 1) if old[i-1] == 1 and old[i] == 1 and old[i + 1] == 1]
            if len(choices)>0:
                choice = np.random.choice(choices)
                splitting_point = choice
                new[splitting_point]=0
        
        if (coin>=0.2) and (coin<0.4): #right removal
            choices = [i for i in range(0,len(old)-1) if old[i] == 1 and old[i + 1] == 0]
            if len(choices)>0:
                choice = np.random.choice(choices)
                removing_point = choice
                new[removing_point]=0
        
        if (coin>=0.4) and (coin<0.6): #left removal
            choices = [i+1 for i in range(0,len(old)-1) if old[i] == 0 and old[i + 1] == 1]
            if len(choices)>0:
                choice = np.random.choice(choices)
                removing_point = choice
                new[removing_point]=0
        
        if (coin>=0.6) and (coin<0.8): #right add
            choices = [i for i in range(1,len(old)) if old[i-1] == 1 and old[i] == 0]
            if len(choices)>0:
                choice = np.random.choice(choices)
                adding_point = choice
                new[adding_point]=1

        if coin>=0.8: #left add
            choices = [i for i in range(0,len(old)-1) if old[i] == 0 and old[i + 1] == 1]
            if len(choices)>0:
                choice = np.random.choice(choices)
                adding_point = choice
                new[adding_point]=1

        if sum(old)==0:
            choice = np.random.choice(range(len(old)))
            new[choice]=1
        return new
        
    
    def acceptance_rate(self,current,new):   
        prior = self.PT0(current)
        prior_new = self.PT0(new)
        likl = posterior(self.T,current,self.x,self.y,self.K,self.sigma).likelihood()
        likl_new =posterior(self.T,new,self.x,self.y,self.K,self.sigma).likelihood()
        return (likl_new*prior_new)/(likl*prior)
            
    
    def PT0(self,current): 
        r=runs(current)          
        return mp.exp(-r*self.alpha)


# Regular Smoothing

In [6]:
sigma = 10
T = np.linspace(1, 288, 288)
K = np.fromfunction(np.vectorize(lambda s, t: prior_kernel(T[s], T[t])), (T.shape[0], T.shape[0]), dtype=int)

In [7]:
np.linalg.det(K)

1.7462173683722895e-21

In [8]:
df = pd.read_csv('average_curves.csv', index_col='idx')
df 

,0,1,2,3,4,5,6,7,8,9,...,150,151,152,153,154,155,156,157,158,159
idx,,,,,,,,,,,,,,,,,,,,,
pat_number,0.000000,1.000000,2.000000,3.000000,4.000000,5.000000,6.000000,7.000000,8.000000,9.000000,...,150.000000,151.000000,152.000000,153.000000,154.000000,155.000000,156.000000,157.000000,158.000000,159.000000
severity,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000
constancy,1.000000,1.000000,2.000000,1.000000,2.000000,2.000000,1.000000,1.000000,1.000000,1.000000,...,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000
age,58.000000,59.000000,62.000000,65.000000,65.000000,70.000000,66.000000,74.000000,58.000000,74.000000,...,68.000000,49.000000,51.000000,66.000000,45.000000,74.000000,67.000000,59.000000,34.000000,72.000000
gender,1.000000,2.000000,2.000000,1.000000,1.000000,2.000000,1.000000,1.000000,1.000000,1.000000,...,2.000000,2.000000,1.000000,1.000000,1.000000,1.000000,1.000000,2.000000,1.000000,2.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
283,-0.981659,-1.076615,-1.332550,-1.058805,-0.923807,-1.109480,-1.743754,-1.184359,-1.359995,-1.107242,...,-1.485589,-1.074087,-1.003103,-0.996204,-1.740179,-1.447827,-1.043098,-1.684661,-1.077221,-1.393487
284,-0.993531,-1.078280,-1.332549,-1.044116,-0.923762,-1.109441,-1.743714,-1.217882,-1.360021,-1.116414,...,-1.486356,-1.074077,-1.002612,-0.994606,-1.740226,-1.445859,-1.042748,-1.684662,-1.077203,-1.393533
285,-0.997926,-1.079618,-1.332558,-1.019142,-0.923818,-1.109399,-1.743523,-1.224489,-1.360155,-1.127783,...,-1.487368,-1.074091,-1.003404,-0.994287,-1.740254,-1.440550,-1.042608,-1.684662,-1.077191,-1.393537


In [9]:
idxs = np.random.permutation(range(159))[:20]
x = df.iloc[7:, idxs].T.to_numpy()
y = df.iloc[1, idxs].T.to_numpy()

In [10]:
y23 = np.copy(y)
y23[y == 1] = 0
y23[y == 2] = 1
y23[y == 3] = 1
np.unique(y23)

array([0., 1.])

In [11]:
chain = T0_est(T, x, y23, K, sigma, 4, 500, 0).chain_builder()

Acceptance Rate = 2855.3902224429711632722244160104334423730342669702

1/500 in findind T0...

Current Number of Points in T0: 145
Acceptance Rate = 2780.4261824053068422383730570734083800313224210458

2/500 in findind T0...

Current Number of Points in T0: 144
Acceptance Rate = 0.00034955697752271496423878970021297649011203052090011

3/500 in findind T0...

Current Number of Points in T0: 144
Acceptance Rate = 3029.1604487248935526990794917413371741545905119744

4/500 in findind T0...

Current Number of Points in T0: 145
Acceptance Rate = 2259.1025168432975505007085493412423925337277456085

5/500 in findind T0...

Current Number of Points in T0: 144
Acceptance Rate = 3154.492301909957808444848947587618024958240721133

6/500 in findind T0...

Current Number of Points in T0: 143
Acceptance Rate = 0.89614619209721132850709946925408394727918668875867

7/500 in findind T0...

Current Number of Points in T0: 143
Acceptance Rate = 0.00035328878914380549585431990162078352432919071058708

8/50

Acceptance Rate = 0.00032510531556705583964400153625812136097399330127511

63/500 in findind T0...

Current Number of Points in T0: 146
Acceptance Rate = 2801.7506864497322555377449387674075830633576577118

64/500 in findind T0...

Current Number of Points in T0: 145
Acceptance Rate = 2973.9020126764478880631751413281454250077286767848

65/500 in findind T0...

Current Number of Points in T0: 146
Acceptance Rate = 0.00033056329637890980782048833882853935005444504993122

66/500 in findind T0...

Current Number of Points in T0: 146
Acceptance Rate = 0.00032010917792526526233413512829017913384252826609579

67/500 in findind T0...

Current Number of Points in T0: 146
Acceptance Rate = 0.00032907799943126917113425714649120383196035651512558

68/500 in findind T0...

Current Number of Points in T0: 146
Acceptance Rate = 3353.243667988405985385036094592718648971962417661

69/500 in findind T0...

Current Number of Points in T0: 147
Acceptance Rate = 3001.80612770247752252737192112525553313115

Acceptance Rate = 1.0331841524730671013085348855385566617512424092978

125/500 in findind T0...

Current Number of Points in T0: 154
Acceptance Rate = 0.00032272693923063203970055464805187873476109611527004

126/500 in findind T0...

Current Number of Points in T0: 154
Acceptance Rate = 2417.2416412806009870465944060827589386991095885055

127/500 in findind T0...

Current Number of Points in T0: 153
Acceptance Rate = 2884.2895708002430855574228993798769270733335656057

128/500 in findind T0...

Current Number of Points in T0: 154
Acceptance Rate = 2651.1110271264227224866548235483650557189113097397

129/500 in findind T0...

Current Number of Points in T0: 155
Acceptance Rate = 3023.5125498886049963092214575447796013997260826017

130/500 in findind T0...

Current Number of Points in T0: 154
Acceptance Rate = 2748.0788565929587098908500315915512399409662053831

131/500 in findind T0...

Current Number of Points in T0: 155
Acceptance Rate = 3049.147967265430046701908007937429565557510766

Acceptance Rate = 1.0814095252281932706703457576854444749048277981639

187/500 in findind T0...

Current Number of Points in T0: 165
Acceptance Rate = 0.98260794203798250068074428240215469437970838725552

188/500 in findind T0...

Current Number of Points in T0: 165
Acceptance Rate = 0.00031071309750524286418498153594616994537832183406745

189/500 in findind T0...

Current Number of Points in T0: 165
Acceptance Rate = 1.1428615306106450699083678930385650981798830798845

190/500 in findind T0...

Current Number of Points in T0: 166
Acceptance Rate = 1.0640884131064836200383597486501450890355847363473

191/500 in findind T0...

Current Number of Points in T0: 167
Acceptance Rate = 0.92635791523686217347749675994464285280672844356726

192/500 in findind T0...

Current Number of Points in T0: 167
Acceptance Rate = 0.99169017959331473430504614462865250147147645251273

193/500 in findind T0...

Current Number of Points in T0: 167
Acceptance Rate = 3277.481658029030644836167089304367711930521

Acceptance Rate = 1.2015701074994746253549509888454728764948430898461

248/500 in findind T0...

Current Number of Points in T0: 178
Acceptance Rate = 0.00036924840299263333486242598991775484519394366393059

249/500 in findind T0...

Current Number of Points in T0: 178
Acceptance Rate = 1.0449210698609346774343580991167966826057186994466

250/500 in findind T0...

Current Number of Points in T0: 177
Acceptance Rate = 0.00033599405589584207126809021453666086244577696133281

251/500 in findind T0...

Current Number of Points in T0: 177
Acceptance Rate = 0.98265802840660784270267713395863070220886487276164

252/500 in findind T0...

Current Number of Points in T0: 177
Acceptance Rate = 0.9180657168740074815544388353381031970463155191304

253/500 in findind T0...

Current Number of Points in T0: 177
Acceptance Rate = 1.639293829763869950516725297760468698332463517033

254/500 in findind T0...

Current Number of Points in T0: 176
Acceptance Rate = 0.68670798574368335926887056337626701224777

Acceptance Rate = 0.9865324693066396107139610487439288488755764154218

310/500 in findind T0...

Current Number of Points in T0: 191
Acceptance Rate = 1.3517560022455777863463840635913380174833027995949

311/500 in findind T0...

Current Number of Points in T0: 190
Acceptance Rate = 1.1372862022680438977357715484873031373810375948882

312/500 in findind T0...

Current Number of Points in T0: 191
Acceptance Rate = 1.3386013935933842487524216728823932562776277435957

313/500 in findind T0...

Current Number of Points in T0: 192
Acceptance Rate = 0.97562026185367839732484004383512493436241937090988

314/500 in findind T0...

Current Number of Points in T0: 192
Acceptance Rate = 0.9920361169139979122127229975310553469885727668688

315/500 in findind T0...

Current Number of Points in T0: 193
Acceptance Rate = 0.74662226512429896783096334027416046708184210878744

316/500 in findind T0...

Current Number of Points in T0: 193
Acceptance Rate = 1.08283060665494700424104426358840179993775564045

Acceptance Rate = 0.76996033010633583799094435272916938802918196574092

371/500 in findind T0...

Current Number of Points in T0: 199
Acceptance Rate = 0.87424002381813788951887340330984750878450858547152

372/500 in findind T0...

Current Number of Points in T0: 199
Acceptance Rate = 2985.7873696300942325753628956227574709279207016195

373/500 in findind T0...

Current Number of Points in T0: 200
Acceptance Rate = 0.76970138448472150229152472226509992533351628013903

374/500 in findind T0...

Current Number of Points in T0: 200
Acceptance Rate = 1.0113599408839172471438984025416155529742807105091

375/500 in findind T0...

Current Number of Points in T0: 199
Acceptance Rate = 0.00028559098143687393065068332063565551855463071910634

376/500 in findind T0...

Current Number of Points in T0: 199
Acceptance Rate = 0.92758307159653583385302364907907263664584928827553

377/500 in findind T0...

Current Number of Points in T0: 199
Acceptance Rate = 0.73210885960108063460818601230751998039390

Acceptance Rate = 0.95886546815888919909602333821421255558504076010057

433/500 in findind T0...

Current Number of Points in T0: 214
Acceptance Rate = 0.73203397340468599132833603507608050725101560502133

434/500 in findind T0...

Current Number of Points in T0: 214
Acceptance Rate = 1.3287580264346422758477186620355464911592001635453

435/500 in findind T0...

Current Number of Points in T0: 215
Acceptance Rate = 0.93303446342075067872820094246321304562554764294981

436/500 in findind T0...

Current Number of Points in T0: 215
Acceptance Rate = 0.9608963492005825527598875675281723142347000617192

437/500 in findind T0...

Current Number of Points in T0: 215
Acceptance Rate = 0.93303446342075067872820094246321304562554764294981

438/500 in findind T0...

Current Number of Points in T0: 215
Acceptance Rate = 0.00030561165944156111117433507654480521585935088831481

439/500 in findind T0...

Current Number of Points in T0: 215
Acceptance Rate = 0.70602502610652264083120947266863611426730

Acceptance Rate = 0.96242801983198600452389448108507035029532671024134

495/500 in findind T0...

Current Number of Points in T0: 228
Acceptance Rate = 0.74083838943186798200050439392513484399824205291987

496/500 in findind T0...

Current Number of Points in T0: 228
Acceptance Rate = 1.0117735708466886995122144505179927129173060456241

497/500 in findind T0...

Current Number of Points in T0: 229
Acceptance Rate = 0.68497194045633626463861558438643083823472107385651

498/500 in findind T0...

Current Number of Points in T0: 229
Acceptance Rate = 0.74792130990363322294283767782101951531164293936151

499/500 in findind T0...

Current Number of Points in T0: 229
Acceptance Rate = 0.91139050189513839517902959963316253220872751471808

500/500 in findind T0...

Current Number of Points in T0: 229


In [12]:
T0_hat, counts = stats.mode(chain[200:], axis=0)

C:\Users\Michele\AppData\Local\Temp\ipykernel_9664\3580705632.py:1: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  T0_hat, counts = stats.mode(chain[200:], axis=0)


In [13]:
T0_hat[0]

array([0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1,
       1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1,
       1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1,
       1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1,
       1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0,
       0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1])

In [14]:
# 20 data points, alpha=2:
# array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       #1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       #1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       #1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1,
       #1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
       #1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1,
       #1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0,
       #0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       #1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       #1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       #1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0,
       #0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1,
       #1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       #0, 0])

In [15]:
fitted = posterior(T,T0_hat[0],x,y23,K,sigma).fitted_values()
logit_fitted = scipy.special.expit(fitted)
    
labels = np.zeros(len(logit_fitted))
ones= np.where(logit_fitted > 0.5)
labels[ones] = np.ones(len(ones)) 

In [16]:
sum(labels==y23)/len(labels)

0.8